# Работа с метриками по SBER

### Импортруемы библиотеки

In [1]:
import pandas as pd
import re
pd.options.mode.chained_assignment = None
import datetime
import numpy as np
import xlrd
import glob
from sys import exit
import sys
link_to_table = 'https://docs.google.com/spreadsheets/d/1A1TyZPeq9r2HiDi-CHfmTfRpuAQYToib1vqyTx4ZyRE/edit#gid=0'
df = pd.read_excel('/'.join(link_to_table.split('/')[:-1])+'/export?format=xlsx')
PATH, df = [dict(df.loc[x]) for x in list(df.index)], None
i = 0
while len([py for py in glob.glob(f"{PATH[i]['path_downloads']}*.xlsx")]) == 0:
    i +=1
path_downloads = PATH[i]['path_downloads']
path_algoritm = PATH[i]['path_algoritm']
sys.path.append(f'{path_algoritm}Модули')
import constants_for_marketplace_metrics_dags as const
import working_with_marketplace_metrics_dags as wwm
import maslow_working_module as mwm
import sb_markups

#### Остатки и цены

In [2]:
## Кабинет Интернет-революция - Настройки - Магазины - Интернет-революция - Скачать
path_new_stocks_and_price_sber = path_downloads + \
'20511_02-04-2024-07-33-51-431_c6b40db6-a66c-4381-ab9d-6715b1c7cec7 (4).xlsx'

#### Заказы

In [3]:
## Кабинет Интернет-революция - FBS - Все заказы - Нужная дата - Скачать 
if mwm.dt_now.date().isoweekday() == 1:
    P = [
         path_downloads + f"заказы-{mwm.dt_now.date().strftime('%Y-%m-%d')}-все.xlsx", 
         path_downloads + f"заказы-{mwm.dt_now.date().strftime('%Y-%m-%d')}-все (1).xlsx", 
         path_downloads + f"заказы-{mwm.dt_now.date().strftime('%Y-%m-%d')}-все (2).xlsx"
        ]
else:
    P = [path_downloads + f"заказы-{mwm.dt_now.date().strftime('%Y-%m-%d')}-все.xlsx"]

#### Готовые связки (остатки)

In [4]:
## Кабинет Интернет-революция - Ассортимент - Готовые связки - Скачать
path_dop_name = path_downloads + f"Готовые связки-{mwm.dt_now.date().strftime('%Y-%m-%d')}.xlsx"

#### Акции

In [5]:
## Файлы по акциям
acc_name = [
'Федеральное промо: Дачная распродажа_01.04 - 14.04',
'Событийное промо: Всемирный день здоровья_22.03-09.04',
'ЗОО Распродажа апреля',
'Сад и Дача 2. Выгоды на товары для подготовки участка к сезону, отдыха и стройки. Скидки от 10%',
'Летние виды спорта_готовимся к сезону_22.03-14.04_Скидка от 10%',
'Суперцены на бытовую технику и электронику',
'Fashion_29.03-11.04_Бижутерия, аксессуары для волос',
'Совместный кешбек - до 50% за счёт МегаМаркета 21.03 - 27.03',
'Готовим дома 01.04-30.04_Товары для сервировки стола и приготовления пищи_скидка от 15% Копия',
'Федеральное промо: Дачная распродажа_15.04 - 25.04',
'Fashion_12.04-25.04_Пора менять гардероб',
'ЗОО Распродажа апреля',
'Время обновлений. Все для ремонта_02.04-31.05_Сикдка от 10%',
'Событийное промо: Пасха_18.04-05.05',
'Скидки на российскую косметику 08.04-21.04',
'Распродажа тренажеров и товаров для фитнеса_15.04-12.05_Скидка от 15%',
]


acc_path = [
'11.03.2024_08.38_goods_for_promotion.xlsx', 
'11.03.2024_08.39_goods_for_promotion.xlsx',
'15.03.2024_09.58_goods_for_promotion.xlsx',
'18.03.2024_07.10_goods_for_promotion.xlsx',
'18.03.2024_07.11_goods_for_promotion.xlsx',
'18.03.2024_07.12_goods_for_promotion.xlsx',
'18.03.2024_07.13_goods_for_promotion.xlsx',
'18.03.2024_06.24_goods_for_promotion.xlsx',
'26.03.2024_06.41_goods_for_promotion.xlsx',
'26.03.2024_06.42_goods_for_promotion.xlsx',
'26.03.2024_06.43_goods_for_promotion.xlsx',
'28.03.2024_07.27_goods_for_promotion.xlsx',
'28.03.2024_07.24_goods_for_promotion.xlsx',
'28.03.2024_07.25_goods_for_promotion.xlsx',
'29.03.2024_06.30_goods_for_promotion.xlsx',
'29.03.2024_06.31_goods_for_promotion.xlsx',
]

In [6]:
new_stocks_and_price_sber = pd.read_excel(path_new_stocks_and_price_sber)

dop_name = pd.read_excel(path_dop_name)
dop_name['Артикул Мегамаркет'] = dop_name['Артикул Мегамаркет'].astype(str)



# фйал с остатками 2.0
name_new_stocks_sber_v2 = ''

if name_new_stocks_sber_v2 != '':

    new_stocks_sber_v2 = pd.read_excel(path_downloads + name_new_stocks_sber_v2,
                                       sheet_name='Список товаров',
                                       usecols=['offer_id', 'instock'],
                                       skiprows=1,
                                       dtype={'offer_id': 'int',
                                              'instock': 'int'})\
                        .rename(columns={'instock': 'Остаток'})\
                        .merge(dop_name[['Код товара продавца (offerId)', 'Артикул Мегамаркет']]\
                               .rename(columns={'Код товара продавца (offerId)': 'offer_id', 
                                                'Артикул Мегамаркет': 'Артикул'}),
                               on='offer_id', 
                               how='left')[['Артикул', 'Остаток']]
else:
    new_stocks_sber_v2 = dop_name[['Артикул Мегамаркет', 'Остаток на складе']]\
                         .rename(columns={'Артикул Мегамаркет': 'Артикул', 'Остаток на складе': 'Остаток'})\
                         .drop_duplicates()\
                         .reset_index(drop=True).copy()

    name_new_stocks_sber_v2 = 'из "dop_name"'

    new_stocks_sber_v2 = new_stocks_sber_v2[new_stocks_sber_v2['Остаток'].astype(str)!='-']\
    .reset_index(drop=True)
    new_stocks_sber_v2['Остаток'] = new_stocks_sber_v2['Остаток'].astype(int)

reestr = wwm.Reestr()

/Users/dmitriymaslov/anaconda3/envs/work_venv/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


#### Метрики

In [7]:
# Ретроданные

path_sber_orders_metrics = path_algoritm + 'SBER_ORDER/sber_orders_metrics.xlsx'

sber_orders_metrics = pd.read_excel(path_sber_orders_metrics)




# Остатки

path_stocks_sber_metrics = path_algoritm + 'SBER_ORDER/stocks_sber_metrics.xlsx'

stocks_sber_metrics = pd.read_excel(path_stocks_sber_metrics)





# Цены

path_sber_price_metrics = path_algoritm + 'SBER_ORDER/sber_price_metrics.xlsx'

sber_price_metrics = pd.read_excel(path_sber_price_metrics)

In [8]:
def pda(data):
    
    if 'Артикул' in data.columns:
        print(f"Уникальных артикулов: {len(list(set(data['Артикул'])))}")
        print(f"Всего артикулов: {len(list(data['Артикул']))}")
        print()
        print()
        print()
        data = data.reset_index(drop=True)
        for i in range(len(data)):
            art = data.loc[i]['Артикул']
            if len(data[data['Артикул']==art]) > 1:
                display(data[data['Артикул']==art])
    else:
        None

In [9]:
def Apocalypse():
    
    sber_orders_metrics = pd.read_excel(path_sber_orders_metrics)
    sber_orders_metrics = sber_orders_metrics.iloc[:, :-1]
    sber_orders_metrics.to_excel(path_sber_orders_metrics, index=False)
    
    stocks_sber_metrics = pd.read_excel(path_stocks_sber_metrics)
    stocks_sber_metrics = stocks_sber_metrics.iloc[:, :-1]
    stocks_sber_metrics.to_excel(path_stocks_sber_metrics, index=False)
    
    sber_price_metrics = pd.read_excel(path_sber_price_metrics)
    sber_price_metrics = sber_price_metrics.iloc[:, :-1]
    sber_price_metrics.to_excel(path_sber_price_metrics, index=False)

In [10]:
for path in [path_sber_orders_metrics, path_stocks_sber_metrics, path_sber_price_metrics]:
    data = pd.read_excel(path, dtype={'Артикул': 'str'})
    data = data[data['Артикул'].astype(str)!='nan'].reset_index(drop=True)
    data.to_excel(path, index=False)
    
data = None

In [11]:
# Apocalypse()

In [12]:
# nam = pd.read_excel(path_downloads + 'Реестр (6).xlsx',
#                     sheet_name='Названия')
# nam = nam.astype(str)

# def df_rename(data, nam, column_name):
    
#     data['Артикул'] = data['Артикул'].astype(str)
    
#     for i in range(len(data)):
#         art = data.loc[i]['Артикул']
#         if art in list(set(nam['Артикул'])):
#             data[column_name].loc[i] = nam[nam['Артикул']==art]['Название'].values[0]
#         else:
#             None

#     return data

In [13]:
# Apocalypse()

In [14]:
# sber_orders_metrics = sber_orders_metrics.iloc[:, :-3]

In [15]:
sber_orders_metrics

,Артикул,Название,Номенклатура,TM,20-February-2023,21-February-2023,22-February-2023,23-February-2023,24-February-2023,25-February-2023,...,29-March-2024,30-March-2024,31-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,06-April-2024,07-April-2024
0,20501,Массажер для шеи и плеч,11111,SSY,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,A5501,Камень точильный 6000/1000,99582,Shiny Kitchen,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,1
2,A5502,Камень точильный 400/1000 и 3000/8000,99093,Shiny Kitchen,0,0,0,0,0,0,...,0,0,0,0,0,1,2,0,0,0
3,G0155,"Сумка для инструментов, форма 1",11006,SSY,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,G0156,"Сумка для инструментов, форма 2",12005,SSY,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985,50187,"Швабра для окон, длинная",25479,Birdhouse,0,0,0,0,0,0,...,0,2,0,0,0,0,0,0,0,0
986,G0708,"Тактический ошейник, бежевый, XL",25500,SSY,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
987,27751,"Утяжелители спортивные черные, 2x2,0 кг.",25406,Shark Fit,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
988,G0284,"Наколенники с медной нитью, L",25526,Shark Fit,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [16]:
# файл с себестоимостью (общий)

path_cost_price = path_algoritm + 'ОБЩИЕ ПРИЗНАКИ ДЛЯ МЕТРИК/Себестоимость /cost_price 2.0.xlsx'

cp = pd.read_excel(path_cost_price)

In [17]:
# файл с признаком "РЦ" (общий)

path_rc = path_algoritm + 'ОБЩИЕ ПРИЗНАКИ ДЛЯ МЕТРИК/РЦ/РЦ_обработанные.xlsx'

rc = pd.read_excel(path_rc)

In [18]:
# файл с признаком "В резерве" (общий)

path_vr = path_algoritm + 'ОБЩИЕ ПРИЗНАКИ ДЛЯ МЕТРИК/В резерве/В резерве_обработанные.xlsx'

In [19]:
# ## файл со статусами Сбер

# path_st = path_algoritm + 'SBER_ORDER/Статусы/Статусы.xlsx'
# st = pd.read_excel(path_st)
# st['Артикул'] = st['Артикул'].astype(str)


# файл со Статусами


path_st = wwm.path_st
st = pd.read_excel(path_st)
st['Артикул'] = st['Артикул'].astype(str)

In [20]:
st

,Артикул,Статус
0,16920,текущий
1,16930,текущий
2,16950,сезонка
3,18704,отказ
4,18705,отказ
...,...,...
1303,50976,текущий Дарья
1304,g0901,текущий Дарья
1305,50458,товары в пути
1306,50459,товары в пути


In [21]:
# Тарифная ставка 

# path_tf = path_downloads + 'Готовые связки-2023-02-21.xlsx'
path_tf = path_dop_name

tf = pd.read_excel(path_tf)

In [22]:
def dt_col(data):
    date_col = []
    for row in data.columns:
        try:
            pd.to_datetime(row)
            date_col.append(row)
        except:
            None
    return date_col

In [23]:
def letter_substitution(data):
    
    data = data.rename(columns={data.columns[0]: 'Артикул'})
    data['Артикул'] = data['Артикул'].astype(str)
    data = data.reset_index(drop=True)
    
    fa = list(filter(lambda x: re.compile('[^а-яА-Я ]').sub('', x), list(data['Артикул'])))

    if len(fa) > 0:

        Q = {'е': 'e', 
             'Е': 'E', 
             'Т': 'T', 
             'у': 'y', 
             'о': 'o', 
             'О': 'O', 
             'р': 'p', 
             'Р': 'P', 
             'а': 'a', 
             'А': 'A', 
             'Н': 'H',
             'К': 'K',
             'х': 'x',
             'Х': 'X',
             'с': 'c',
             'С': 'C',
             'В': 'B',
             'М': 'M'}

        for i in range(len(data)):

            for row in list(Q.keys()):

                if row in data.loc[i]['Артикул']:

                    data['Артикул'].loc[i] = data.loc[i]['Артикул'].replace(row, Q[row])


    fa2 = list(filter(lambda x: re.compile('[^а-яА-Я ]').sub('', x), list(data['Артикул'])))

    if len(fa2) == 0:

        return data



In [24]:
new_stocks_and_price_sber

,id,Доступность товара,Категория,Производитель (Бренд),Артикул,Модель,Название,Цена(руб),Старая цена(руб),Остаток,НДС,Штрихкод,Ссылка на картинку,Описание,Ссылка на товар на сайте магазина,Время заказа До,Дней на отгрузку
0,offer_id,available,category,vendor,vendor_code,model,name,price,old_price,instock,vat,barcode,picture,description,url,order-before,days
1,25161,Доступен,Комплекты разных столовых приборов,Shiny Kitchen,16970,Приборы для сервировки стола,Набор столовых приборов Shiny Kitchen Приборы ...,1370,2360,0,20,110100584802,https://disk.yandex.ru/i/tDoWbbeH7tBubQ,Набор столовых приборов из 24 предметов - это ...,NaN,NaN,NaN
2,422,Доступен,Спонж для макияжа,REVOLUT,19802,"Спонж для нанесения косметических средств ""Яй...","Спонж для макияжа, REVOLUT, Спонж для косметич...",190,330,0,20,110069312119,https://disk.yandex.ru/i/PFwCtSFl6ilo8A,Спонжи для макияжа — настоящий мастхэв в любой...,NaN,NaN,NaN
3,1151,Доступен,Формы и пакеты для льда,Sugar Day,A1202,"Формочка для леденцов с крышкой, Контейнер для...","Sugar Day, Силиконовая форма для льда и шокола...",200,390,0,20,110069349979,https://disk.yandex.ru/i/zDR32su7XGMZGQ,"Форма для льда — та самая вещь, которая есть в...",NaN,NaN,NaN
4,25215,Доступен,Бигуди,REVOLUT,29311,Атласный керлик для локонов,"Бигуди для волос мягкие, REVOLUT, Атласный кер...",345,600,0,20,110106264456,https://disk.yandex.ru/i/CmG_w3yvPmqsYg,"Роскошные локоны — это легко, если у вас есть ...",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1093,25589,Доступен,Органайзеры между передними сиденьями,Flex Drive,G0337,Карман в автомобиль,Органайзер автомобильный между сиденьями в маш...,1212,2431,0,20,4662413729051,https://ir.ozone.ru/s3/multimedia-l/wc1000/683...,Зачем захламлять переднюю панель в машине всяк...,NaN,NaN,NaN
1094,25590,Доступен,Полка для ванной,Birdhouse,B0164,без сверления пластиковые,Полка для ванной Birdhouse без сверления пласт...,1060,2430,0,20,4664704067644,https://ir.ozone.ru/s3/multimedia-o/wc1000/683...,Прозрачные полочки для ванной — мечта любителе...,NaN,NaN,NaN
1095,25591,Доступен,Женские косметички,Travel Friendly,50372,дорожная с отделениями на молнии,осметичка Travel Friendly дорожная с отделения...,635,840,0,20,4672624409857,https://ir.ozone.ru/s3/multimedia-1-h/wc1000/6...,Вместительная косметичка дорожная — это незаме...,NaN,NaN,NaN
1096,25592,Доступен,Интерьерные наклейки,Birdhouse,50976,Зеркальные наклейки,"Наклейки на стену для декора Birdhouse, Зеркал...",399,501,0,20,4673660046877,https://ir.ozone.ru/s3/multimedia-c/wc1000/662...,"Все знают, что зеркальная поверхность визуальн...",NaN,NaN,NaN


In [25]:
new_stocks_and_price_sber = new_stocks_and_price_sber[['Артикул', 'Модель', 'id', 'Производитель (Бренд)', 'Цена(руб)', 'Остаток']].iloc[1:, :].reset_index(drop=True)
new_stocks_and_price_sber.columns = ['Артикул', 'Название', 'Номенклатура', 'TM', 'Цена', 'Остаток']
new_stocks_and_price_sber[['Артикул', 'Название', 'TM']] = new_stocks_and_price_sber[['Артикул', 'Название', 'TM']].astype(str)
new_stocks_and_price_sber[['Номенклатура', 'Цена', 'Остаток']] = new_stocks_and_price_sber[['Номенклатура', 'Цена', 'Остаток']].astype(int)
new_stocks_and_price_sber = new_stocks_and_price_sber.drop_duplicates().reset_index(drop=True)
new_stocks_and_price_sber = letter_substitution(new_stocks_and_price_sber)
new_stocks_and_price_sber['Артикул'] = new_stocks_and_price_sber['Артикул'].astype(str)

In [26]:
new_stocks_and_price_sber[new_stocks_and_price_sber['Артикул']=='28812']

,Артикул,Название,Номенклатура,TM,Цена,Остаток
63,28812,Суппорт на запястье 2 шт,25280,Shark Fit,580,0


In [27]:
new_stocks_and_price_sber

,Артикул,Название,Номенклатура,TM,Цена,Остаток
0,16970,Приборы для сервировки стола,25161,Shiny Kitchen,1370,0
1,19802,"Спонж для нанесения косметических средств ""Яй...",422,REVOLUT,190,0
2,A1202,"Формочка для леденцов с крышкой, Контейнер для...",1151,Sugar Day,200,0
3,29311,Атласный керлик для локонов,25215,REVOLUT,345,0
4,17013,"Выделители текста пастельные, Набор 6 шт",25341,Parchment,180,0
...,...,...,...,...,...,...
1092,G0337,Карман в автомобиль,25589,Flex Drive,1212,0
1093,B0164,без сверления пластиковые,25590,Birdhouse,1060,0
1094,50372,дорожная с отделениями на молнии,25591,Travel Friendly,635,0
1095,50976,Зеркальные наклейки,25592,Birdhouse,399,0


In [28]:
R = pd.to_datetime(path_new_stocks_and_price_sber.split('Загрузки Маслов/')[1].split('_')[1][:10], format='%d-%m-%Y').strftime('%d-%B-%Y')

In [29]:
R

'02-April-2024'

In [30]:
# R = '23-March-2023'

In [31]:
if R not in list(sber_price_metrics.columns):
    sber_price_metrics = sber_price_metrics.merge(new_stocks_and_price_sber[['Артикул', 'Цена']], on='Артикул', how='outer')
    sber_price_metrics = sber_price_metrics.rename(columns={'Цена': R})
    sber_price_metrics = sber_price_metrics.fillna(0)
    for col in sber_price_metrics.columns[1:]:
        sber_price_metrics[col] = sber_price_metrics[col].astype(float)
else:
    None    

In [32]:
sber_price_metrics['Артикул'] = sber_price_metrics['Артикул'].astype(str)

In [33]:
new_stocks_and_price_sber

,Артикул,Название,Номенклатура,TM,Цена,Остаток
0,16970,Приборы для сервировки стола,25161,Shiny Kitchen,1370,0
1,19802,"Спонж для нанесения косметических средств ""Яй...",422,REVOLUT,190,0
2,A1202,"Формочка для леденцов с крышкой, Контейнер для...",1151,Sugar Day,200,0
3,29311,Атласный керлик для локонов,25215,REVOLUT,345,0
4,17013,"Выделители текста пастельные, Набор 6 шт",25341,Parchment,180,0
...,...,...,...,...,...,...
1092,G0337,Карман в автомобиль,25589,Flex Drive,1212,0
1093,B0164,без сверления пластиковые,25590,Birdhouse,1060,0
1094,50372,дорожная с отделениями на молнии,25591,Travel Friendly,635,0
1095,50976,Зеркальные наклейки,25592,Birdhouse,399,0


In [34]:
R

'02-April-2024'

In [35]:
if name_new_stocks_sber_v2 != '':
    new_stocks_sber_v2['Артикул'] = new_stocks_sber_v2['Артикул'].astype(str)
    wwm.unique_items(new_stocks_sber_v2)
    if pd.to_datetime(R) < datetime.datetime.now():
        R = datetime.datetime.now().date().strftime('%d-%B-%Y')
        
    if R not in list(stocks_sber_metrics.columns):
        stocks_sber_metrics = stocks_sber_metrics.merge(new_stocks_sber_v2[['Артикул', 'Остаток']], on='Артикул', how='outer')
        stocks_sber_metrics = stocks_sber_metrics.rename(columns={'Остаток': R})
        stocks_sber_metrics = stocks_sber_metrics.fillna(0)
        for col in stocks_sber_metrics.columns[1:]:
            stocks_sber_metrics[col] = stocks_sber_metrics[col].astype(int)
    else:
        None

else:
    
    if R not in list(stocks_sber_metrics.columns):
        stocks_sber_metrics = stocks_sber_metrics.merge(new_stocks_and_price_sber[['Артикул', 'Остаток']], on='Артикул', how='outer')
        stocks_sber_metrics = stocks_sber_metrics.rename(columns={'Остаток': R})
        stocks_sber_metrics = stocks_sber_metrics.fillna(0)
        for col in stocks_sber_metrics.columns[1:]:
            stocks_sber_metrics[col] = stocks_sber_metrics[col].astype(int)
    else:
        None 

In [36]:
stocks_sber_metrics

,Артикул,21-February-2023,28-February-2023,01-March-2023,03-March-2023,06-March-2023,07-March-2023,09-March-2023,13-March-2023,14-March-2023,...,27-March-2024,28-March-2024,29-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,08-April-2024,09-April-2024
0,20501,10,10,10,10,10,0,0,0,0,...,333,333,333,303,302,302,302,302,302,302
1,A5501,10,10,10,10,10,10,10,10,10,...,104,179,178,177,176,176,176,175,174,173
2,A5502,10,10,10,10,10,10,10,10,10,...,178,176,174,173,173,160,159,159,158,150
3,G0155,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,G0156,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1092,50975,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1093,50372,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1094,G0641,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1095,G0337,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
stocks_sber_metrics

,Артикул,21-February-2023,28-February-2023,01-March-2023,03-March-2023,06-March-2023,07-March-2023,09-March-2023,13-March-2023,14-March-2023,...,27-March-2024,28-March-2024,29-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,08-April-2024,09-April-2024
0,20501,10,10,10,10,10,0,0,0,0,...,333,333,333,303,302,302,302,302,302,302
1,A5501,10,10,10,10,10,10,10,10,10,...,104,179,178,177,176,176,176,175,174,173
2,A5502,10,10,10,10,10,10,10,10,10,...,178,176,174,173,173,160,159,159,158,150
3,G0155,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,G0156,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1092,50975,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1093,50372,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1094,G0641,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1095,G0337,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
stocks_sber_metrics

,Артикул,21-February-2023,28-February-2023,01-March-2023,03-March-2023,06-March-2023,07-March-2023,09-March-2023,13-March-2023,14-March-2023,...,27-March-2024,28-March-2024,29-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,08-April-2024,09-April-2024
0,20501,10,10,10,10,10,0,0,0,0,...,333,333,333,303,302,302,302,302,302,302
1,A5501,10,10,10,10,10,10,10,10,10,...,104,179,178,177,176,176,176,175,174,173
2,A5502,10,10,10,10,10,10,10,10,10,...,178,176,174,173,173,160,159,159,158,150
3,G0155,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,G0156,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1092,50975,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1093,50372,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1094,G0641,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1095,G0337,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
sber_price_metrics

,Артикул,21-February-2023,28-February-2023,01-March-2023,03-March-2023,06-March-2023,07-March-2023,09-March-2023,13-March-2023,14-March-2023,...,29-March-2024,30-March-2024,31-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,06-April-2024,07-April-2024
0,20501,3190,3190,3190,3190,3190,3190,3190,3190,3190,...,2240,2240,2240,2240,2240,2240,2240,2240,2240,2240
1,A5501,1799,1799,1799,1799,1799,1799,1799,1799,1799,...,1410,1410,1410,1410,1410,1410,1410,1410,1410,1410
2,A5502,2215,2215,2215,2215,2215,2215,2215,2215,2215,...,2270,2270,2270,2270,2270,2270,2270,2270,2270,2270
3,G0155,3399,3399,3399,3399,3399,3399,3399,3399,3399,...,3399,3399,3399,3399,3399,3399,3399,3399,3399,3399
4,G0156,3599,3599,3599,3599,3599,3599,3599,3599,3599,...,3599,3599,3599,3599,3599,3599,3599,3599,3599,3599
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1092,G0337,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1212,1212,1212,1212,1212,1212
1093,B0164,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1060,1060,1060,1060,1060,1060
1094,50372,0,0,0,0,0,0,0,0,0,...,0,0,0,0,635,635,635,635,635,635
1095,50976,0,0,0,0,0,0,0,0,0,...,0,0,0,0,399,399,399,399,399,399


In [40]:
len(stocks_sber_metrics) - len(list(set(stocks_sber_metrics['Артикул'])))

0

In [41]:
len(sber_price_metrics) - len(list(set(sber_price_metrics['Артикул'])))

0

## Добавление пропущенных дней по ценам

In [42]:
res = [x for x in \
           mwm.date_range(start_date=mwm.dt_now.date() - datetime.timedelta(days=1),
           end_date=pd.to_datetime(wwm.dt_col_s(sber_price_metrics)[-1], format='%d-%B-%Y'))]



/Users/dmitriymaslov/Мой диск/Алгоритмы Маслов/Модули/maslow_working_module.py:25: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  min(start_date, end_date),
/Users/dmitriymaslov/Мой диск/Алгоритмы Маслов/Модули/maslow_working_module.py:26: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  max(start_date, end_date)


In [43]:
for row in res[1:]:
    if row not in sber_price_metrics:
        sber_price_metrics[row] = sber_price_metrics[res[0]]

In [44]:
sber_price_metrics

,Артикул,21-February-2023,28-February-2023,01-March-2023,03-March-2023,06-March-2023,07-March-2023,09-March-2023,13-March-2023,14-March-2023,...,30-March-2024,31-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,06-April-2024,07-April-2024,08-April-2024
0,20501,3190,3190,3190,3190,3190,3190,3190,3190,3190,...,2240,2240,2240,2240,2240,2240,2240,2240,2240,2240
1,A5501,1799,1799,1799,1799,1799,1799,1799,1799,1799,...,1410,1410,1410,1410,1410,1410,1410,1410,1410,1410
2,A5502,2215,2215,2215,2215,2215,2215,2215,2215,2215,...,2270,2270,2270,2270,2270,2270,2270,2270,2270,2270
3,G0155,3399,3399,3399,3399,3399,3399,3399,3399,3399,...,3399,3399,3399,3399,3399,3399,3399,3399,3399,3399
4,G0156,3599,3599,3599,3599,3599,3599,3599,3599,3599,...,3599,3599,3599,3599,3599,3599,3599,3599,3599,3599
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1092,G0337,0,0,0,0,0,0,0,0,0,...,0,0,0,1212,1212,1212,1212,1212,1212,1212
1093,B0164,0,0,0,0,0,0,0,0,0,...,0,0,0,1060,1060,1060,1060,1060,1060,1060
1094,50372,0,0,0,0,0,0,0,0,0,...,0,0,0,635,635,635,635,635,635,635
1095,50976,0,0,0,0,0,0,0,0,0,...,0,0,0,399,399,399,399,399,399,399


In [45]:
new_sber_orders = None
E = None
    
for path_new_sber_orders in P:

    new_sber_orders = pd.read_excel(path_new_sber_orders)

    sber_orders_metrics['Артикул'] = sber_orders_metrics['Артикул'].astype(str)
    new_sber_orders['Артикул'] = new_sber_orders['Артикул'].astype(str)

    new_sber_orders = dop_name[['Артикул Мегамаркет', 'Артикул Мегамаркет (МегамаркетId)']]\
                     .rename(columns={'Артикул Мегамаркет (МегамаркетId)': 'Код товара Мегамаркета'})\
                     .merge(new_sber_orders,
                            on='Код товара Мегамаркета',
                            how='right')
    new_sber_orders['Артикул'] = new_sber_orders['Артикул Мегамаркет']
    new_sber_orders['Артикул']  = new_sber_orders['Артикул'].astype(str)


    if len(new_sber_orders) > 0:
        new_sber_orders = new_sber_orders\
        .query(f"Состояние in {list(filter(lambda x: 'отмен' not in x.lower(), list(new_sber_orders['Состояние'].unique())))}")\
        .reset_index(drop=True)
        try:
            new_sber_orders['Заказ оформлен'] = [pd.to_datetime(x, format='%Y-%m-%d')\
                                              .strftime('%d-%B-%Y') for x in list(new_sber_orders['Заказ оформлен'])]
        except:
            new_sber_orders['Заказ оформлен'] = [pd.to_datetime(x, format='%d.%m.%y')\
                                              .strftime('%d-%B-%Y') for x in list(new_sber_orders['Заказ оформлен'])]
        E = sorted(dict(new_sber_orders['Заказ оформлен'].value_counts()).items(), key=lambda item: item[1])[::-1][0][0]


    #     E = '20-December-2023'

        new_sber_orders = new_sber_orders[['Артикул', 'Кол-во']]
        new_sber_orders.columns = ['Артикул', E]
        new_sber_orders = new_sber_orders.groupby('Артикул')[new_sber_orders.columns[-1]].sum().reset_index(drop=False)

        if E not in list(sber_orders_metrics.columns):

            sber_orders_metrics = sber_orders_metrics.merge(new_sber_orders, on='Артикул', how='outer')
            sber_orders_metrics[E] = sber_orders_metrics[E].fillna(0)

        else:
            None


    else:
        None

    sber_orders_metrics[dt_col(sber_orders_metrics)] = sber_orders_metrics[dt_col(sber_orders_metrics)].fillna(0)
    sber_orders_metrics[dt_col(sber_orders_metrics)] = sber_orders_metrics[dt_col(sber_orders_metrics)].astype(int)
    
    E = None

In [46]:
wwm.unique_items(sber_orders_metrics)

In [47]:
sber_price_metrics

,Артикул,21-February-2023,28-February-2023,01-March-2023,03-March-2023,06-March-2023,07-March-2023,09-March-2023,13-March-2023,14-March-2023,...,30-March-2024,31-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,06-April-2024,07-April-2024,08-April-2024
0,20501,3190,3190,3190,3190,3190,3190,3190,3190,3190,...,2240,2240,2240,2240,2240,2240,2240,2240,2240,2240
1,A5501,1799,1799,1799,1799,1799,1799,1799,1799,1799,...,1410,1410,1410,1410,1410,1410,1410,1410,1410,1410
2,A5502,2215,2215,2215,2215,2215,2215,2215,2215,2215,...,2270,2270,2270,2270,2270,2270,2270,2270,2270,2270
3,G0155,3399,3399,3399,3399,3399,3399,3399,3399,3399,...,3399,3399,3399,3399,3399,3399,3399,3399,3399,3399
4,G0156,3599,3599,3599,3599,3599,3599,3599,3599,3599,...,3599,3599,3599,3599,3599,3599,3599,3599,3599,3599
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1092,G0337,0,0,0,0,0,0,0,0,0,...,0,0,0,1212,1212,1212,1212,1212,1212,1212
1093,B0164,0,0,0,0,0,0,0,0,0,...,0,0,0,1060,1060,1060,1060,1060,1060,1060
1094,50372,0,0,0,0,0,0,0,0,0,...,0,0,0,635,635,635,635,635,635,635
1095,50976,0,0,0,0,0,0,0,0,0,...,0,0,0,399,399,399,399,399,399,399


In [48]:
stocks_sber_metrics

,Артикул,21-February-2023,28-February-2023,01-March-2023,03-March-2023,06-March-2023,07-March-2023,09-March-2023,13-March-2023,14-March-2023,...,27-March-2024,28-March-2024,29-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,08-April-2024,09-April-2024
0,20501,10,10,10,10,10,0,0,0,0,...,333,333,333,303,302,302,302,302,302,302
1,A5501,10,10,10,10,10,10,10,10,10,...,104,179,178,177,176,176,176,175,174,173
2,A5502,10,10,10,10,10,10,10,10,10,...,178,176,174,173,173,160,159,159,158,150
3,G0155,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,G0156,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1092,50975,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1093,50372,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1094,G0641,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1095,G0337,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
sber_orders_metrics

,Артикул,Название,Номенклатура,TM,20-February-2023,21-February-2023,22-February-2023,23-February-2023,24-February-2023,25-February-2023,...,30-March-2024,31-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,06-April-2024,07-April-2024,08-April-2024
0,20501,Массажер для шеи и плеч,11111,SSY,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,A5501,Камень точильный 6000/1000,99582,Shiny Kitchen,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,1,0
2,A5502,Камень точильный 400/1000 и 3000/8000,99093,Shiny Kitchen,0,0,0,0,0,0,...,0,0,0,0,1,2,0,0,0,1
3,G0155,"Сумка для инструментов, форма 1",11006,SSY,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,G0156,"Сумка для инструментов, форма 2",12005,SSY,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985,50187,"Швабра для окон, длинная",25479,Birdhouse,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
986,G0708,"Тактический ошейник, бежевый, XL",25500,SSY,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0
987,27751,"Утяжелители спортивные черные, 2x2,0 кг.",25406,Shark Fit,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
988,G0284,"Наколенники с медной нитью, L",25526,Shark Fit,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [50]:
sber_name = new_stocks_and_price_sber[['Артикул', 'Название', 'Номенклатура', 'TM']].copy()
sber_name['Артикул'] = sber_name['Артикул'].astype(str)
sber_orders_metrics['Артикул'] = sber_orders_metrics['Артикул'].astype(str)

In [51]:
for i in range(len(sber_orders_metrics)):
    if sber_orders_metrics.loc[i]['Артикул'] in list(set(sber_name['Артикул'])):
        
        if sber_orders_metrics['Название'].loc[i:i].isna().values[0] == True:
            sber_orders_metrics['Название'].loc[i] = sber_name[sber_name['Артикул']==sber_orders_metrics.loc[i]['Артикул']]['Название'].values[0]
        else:
            None
            
        if sber_orders_metrics['Номенклатура'].loc[i:i].isna().values[0] == True:
            sber_orders_metrics['Номенклатура'].loc[i] = sber_name[sber_name['Артикул']==sber_orders_metrics.loc[i]['Артикул']]['Номенклатура'].values[0]
        else:
            None
            
        if sber_orders_metrics['TM'].loc[i:i].isna().values[0] == True:
            sber_orders_metrics['TM'].loc[i] = sber_name[sber_name['Артикул']==sber_orders_metrics.loc[i]['Артикул']]['TM'].values[0]
        else:
            None
    

In [52]:
sber_name

,Артикул,Название,Номенклатура,TM
0,16970,Приборы для сервировки стола,25161,Shiny Kitchen
1,19802,"Спонж для нанесения косметических средств ""Яй...",422,REVOLUT
2,A1202,"Формочка для леденцов с крышкой, Контейнер для...",1151,Sugar Day
3,29311,Атласный керлик для локонов,25215,REVOLUT
4,17013,"Выделители текста пастельные, Набор 6 шт",25341,Parchment
...,...,...,...,...
1092,G0337,Карман в автомобиль,25589,Flex Drive
1093,B0164,без сверления пластиковые,25590,Birdhouse
1094,50372,дорожная с отделениями на молнии,25591,Travel Friendly
1095,50976,Зеркальные наклейки,25592,Birdhouse


In [53]:
sber_orders_metrics

,Артикул,Название,Номенклатура,TM,20-February-2023,21-February-2023,22-February-2023,23-February-2023,24-February-2023,25-February-2023,...,30-March-2024,31-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,06-April-2024,07-April-2024,08-April-2024
0,20501,Массажер для шеи и плеч,11111,SSY,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,A5501,Камень точильный 6000/1000,99582,Shiny Kitchen,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,1,0
2,A5502,Камень точильный 400/1000 и 3000/8000,99093,Shiny Kitchen,0,0,0,0,0,0,...,0,0,0,0,1,2,0,0,0,1
3,G0155,"Сумка для инструментов, форма 1",11006,SSY,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,G0156,"Сумка для инструментов, форма 2",12005,SSY,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985,50187,"Швабра для окон, длинная",25479,Birdhouse,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
986,G0708,"Тактический ошейник, бежевый, XL",25500,SSY,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0
987,27751,"Утяжелители спортивные черные, 2x2,0 кг.",25406,Shark Fit,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
988,G0284,"Наколенники с медной нитью, L",25526,Shark Fit,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [54]:
sbm = sber_orders_metrics[list(sber_orders_metrics.columns[:4])+dt_col(sber_orders_metrics)[-49:]].copy()

In [55]:
sbp = sber_price_metrics[['Артикул', sber_price_metrics.columns[-1]]]
sbp.columns = ['Артикул', 'Цена']

sbs = stocks_sber_metrics[['Артикул', stocks_sber_metrics.columns[-1]]]
sbs.columns = ['Артикул', 'Остаток']

In [56]:
sbm = sbm.merge(sbp, on='Артикул', how='left')
sbm['Цена'] = sbm['Цена'].fillna(0)
sbm['Цена'] = sbm['Цена'].astype(float)

sbm = sbm.merge(sbs, on='Артикул', how='left')
sbm['Остаток'] = sbm['Остаток'].fillna(0)
sbm['Остаток'] = sbm['Остаток'].astype(int)

In [57]:
sbm = sbm.merge(cp, on='Артикул', how='left')
sbm['Себестоимость'] = sbm['Себестоимость'].fillna(0)
sbm['Себестоимость'] = sbm['Себестоимость'].astype(float)

In [58]:
tf = tf[['Артикул Мегамаркет', 'Тарифная ставка, %']]
tf.columns = ['Артикул', 'тарифная ставка']
tf['Артикул'] = tf['Артикул'].astype(str)
tf = tf.drop_duplicates().reset_index(drop=True)
if 'Z0060' not in list(set(tf['Артикул'])):
    tf.loc[len(tf)] = ['Z0060', tf[tf['Артикул']=='Z0061']['тарифная ставка'].values[0]]
len(tf) - len(list(set(tf['Артикул'])))

0

In [59]:
sbm = sbm.merge(tf, on='Артикул', how='left')
sbm['тарифная ставка'] = sbm['тарифная ставка'].fillna(0)
sbm['тарифная ставка'] = sbm['тарифная ставка'].astype(float)

In [60]:
sbm

,Артикул,Название,Номенклатура,TM,20-February-2024,21-February-2024,22-February-2024,23-February-2024,24-February-2024,25-February-2024,...,03-April-2024,04-April-2024,05-April-2024,06-April-2024,07-April-2024,08-April-2024,Цена,Остаток,Себестоимость,тарифная ставка
0,20501,Массажер для шеи и плеч,11111,SSY,0,0,0,0,0,0,...,0,0,0,0,0,0,2240.0,302,962.76,0.0
1,A5501,Камень точильный 6000/1000,99582,Shiny Kitchen,0,0,0,0,0,0,...,0,0,0,0,1,0,1410.0,173,536.15,0.0
2,A5502,Камень точильный 400/1000 и 3000/8000,99093,Shiny Kitchen,0,0,0,0,0,1,...,1,2,0,0,0,1,2270.0,150,923.57,0.0
3,G0155,"Сумка для инструментов, форма 1",11006,SSY,0,0,0,0,0,0,...,0,0,0,0,0,0,3399.0,0,1261.64,0.0
4,G0156,"Сумка для инструментов, форма 2",12005,SSY,0,0,0,0,0,0,...,0,0,0,0,0,0,3599.0,0,1124.57,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985,50187,"Швабра для окон, длинная",25479,Birdhouse,0,0,0,0,0,0,...,0,0,0,0,0,0,850.0,5,152.66,0.0
986,G0708,"Тактический ошейник, бежевый, XL",25500,SSY,0,0,0,0,0,0,...,2,0,0,0,0,0,850.0,98,247.18,0.0
987,27751,"Утяжелители спортивные черные, 2x2,0 кг.",25406,Shark Fit,0,0,0,0,0,0,...,0,0,0,0,1,0,1100.0,108,173.19,0.0
988,G0284,"Наколенники с медной нитью, L",25526,Shark Fit,0,0,0,0,0,0,...,0,0,0,0,1,0,1190.0,478,242.36,0.0


## Обновление данных по тарифной ставке сбера в таблице бд

In [61]:
mwm.update_or_create_table_not_date(tf.fillna(0).rename(columns={'тарифная ставка': 'tariff_rate'}), 
                                    'sb_tariff_rate', 
                                    ['Артикул'], 
                                    schema=mwm.ir_schema)

In [62]:
def sbm_nac(data):
    
    def log(y):
        s = None
        if data.loc[y]['Комиссия за логистику, %'] < 30:
            s = 30
        elif data.loc[y]['Комиссия за логистику, %'] > 280:
            s = 280
        else:
            s = data.loc[y]['Комиссия за логистику, %']

        return s
    
    def dost_poc(y):
        s = None
        if data.loc[y]['Комиссия за доставку, %'] < 30:
            s = 30
        elif data.loc[y]['Комиссия за доставку, %'] > 215:
            s = 280
        else:
            s = data.loc[y]['Комиссия за доставку, %']

        return s
    
    data['тарифная ставка в рублях'] = (data['Цена'] / 100) * data['тарифная ставка']
    data['Комиссия за логистику, %'] = (data['Цена'] / 100) * 1
    data['Логистика'] = list(map(lambda x: log(x), list(data.index)))
    data['Комиссия за доставку, %'] = (data['Цена'] / 100) * 4
    data['Доставка до покупателя'] = list(map(lambda x: dost_poc(x), list(data.index)))
    data['Итого логистика'] = data['Логистика'] + data['Доставка до покупателя'] + 10
    data['обработка платежей'] = (data['Цена'] / 100) * 1.5
    data['Цена_ндс'] = data['Цена'] * 0.2 / 1.2
    data['Себестоимость_ндс'] = data['Себестоимость'] * 0.2 / 1.2
    data['тарифная ставка в рублях_ндс'] = data['тарифная ставка в рублях'] * 0.2 / 1.2
    data['Итого логистика_ндс'] = data['Итого логистика'] * 0.2 / 1.2
    data['обработка платежей_ндс'] = data['обработка платежей'] * 0.2 / 1.2
    data['ндс'] = data['Цена_ндс'] - data['Себестоимость_ндс'] - data['тарифная ставка в рублях_ндс'] - data['Итого логистика_ндс'] - data['обработка платежей_ндс']
    data['Цена_ннп'] = data['Цена'] - data['Цена_ндс']
    data['Себестоимость_ннп'] = data['Себестоимость'] - data['Себестоимость_ндс']
    data['тарифная ставка в рублях_ннп'] = data['тарифная ставка в рублях'] - data['тарифная ставка в рублях_ндс']
    data['Итого логистика_ннп'] = data['Итого логистика'] - data['Итого логистика_ндс']
    data['обработка платежей_ннп'] = data['обработка платежей'] - data['обработка платежей_ндс']
    data['налог на пр'] = (data['Цена_ннп'] - data['Себестоимость_ннп'] - data['тарифная ставка в рублях_ннп'] - data['Итого логистика_ннп'] - data['обработка платежей_ннп']) * 0.2
    data['итого налог'] = data['ндс'] + data['налог на пр']
    data['% налога'] = data['итого налог'] * 100 / data['Цена']
    data['прибыль'] = data['Цена'] - data['тарифная ставка в рублях'] - data['Итого логистика'] - data['обработка платежей'] - data['итого налог'] - data['Себестоимость']
    data['Наценка'] = data['прибыль'] * 100 / data['Себестоимость']
    
    return data

In [63]:
sbm_acc = sbm.copy()
sbm = sbm_nac(sbm)

In [64]:
sbm = sbm.merge(rc, on='Артикул', how='left')
sbm['Остаток на РЦ'] = sbm['Остаток на РЦ'].fillna(0)
sbm['Остаток на РЦ'] = sbm['Остаток на РЦ'].astype(int)

In [65]:
sbm = sbm.merge(st, on='Артикул', how='left')
sbm = sbm[list(sbm.columns[:4]) + ['Статус'] + list(sbm.columns[4:-1])]
sbm['Статус'] = sbm['Статус'].fillna('статус не определен')

In [66]:
sbm['Наценка'] = sbm['Наценка'].round(2)

In [67]:
sbm['          '] = None

In [68]:
# sbm = sbm.drop_duplicates().reset_index(drop=True)

In [69]:
len(sbm) - len(list(set(sbm['Артикул'])))

0

In [70]:
datf = sber_orders_metrics

for i in range(len(datf)):
    if len(datf[datf['Артикул']==datf.loc[i]['Артикул']]) > 1:
        display(datf[datf['Артикул']==datf.loc[i]['Артикул']])


In [71]:
sbm_22 = sbm.copy()
sbm = sbm[list(sbm.columns[:5])+dt_col(sbm)+['          ', 'Остаток', 'Цена', 'Наценка', 'Остаток на РЦ']]

In [72]:
sbm.loc[len(sbm)] = ['Итого', None, None, None, None] + list(map(lambda x: sbm[x].sum(), dt_col(sbm))) + [None, sbm['Остаток'].sum(), None, None, sbm['Остаток на РЦ'].sum()] + ([None] * len(list(sbm.columns[sbm.columns.get_loc('Остаток на РЦ')+1:])))

In [73]:
sbm

,Артикул,Название,Номенклатура,TM,Статус,20-February-2024,21-February-2024,22-February-2024,23-February-2024,24-February-2024,...,04-April-2024,05-April-2024,06-April-2024,07-April-2024,08-April-2024,,Остаток,Цена,Наценка,Остаток на РЦ
0,20501,Массажер для шеи и плеч,11111,SSY,текущий,0,0,0,0,0,...,0,0,0,0,0,None,302,2240.0,77.14,605
1,A5501,Камень точильный 6000/1000,99582,Shiny Kitchen,отказ,0,0,0,0,0,...,0,0,0,1,0,None,173,1410.0,94.04,348
2,A5502,Камень точильный 400/1000 и 3000/8000,99093,Shiny Kitchen,отказ,0,0,0,0,0,...,2,0,0,0,1,None,150,2270.0,85.29,303
3,G0155,"Сумка для инструментов, форма 1",11006,SSY,архив,0,0,0,0,0,...,0,0,0,0,0,None,0,3399.0,100.74,0
4,G0156,"Сумка для инструментов, форма 2",12005,SSY,отказ,0,0,0,0,0,...,0,0,0,0,0,None,0,3599.0,132.23,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
986,G0708,"Тактический ошейник, бежевый, XL",25500,SSY,новинка Даша,0,0,0,0,0,...,0,0,0,0,0,None,98,850.0,139.19,197
987,27751,"Утяжелители спортивные черные, 2x2,0 кг.",25406,Shark Fit,текущий Дарья,0,0,0,0,0,...,0,0,0,1,0,None,108,1100.0,318.07,259
988,G0284,"Наколенники с медной нитью, L",25526,Shark Fit,текущий Дарья,0,0,0,0,0,...,0,0,0,1,0,None,478,1190.0,231.66,959
989,G0702,"Тактический ошейник, черный, XL",25494,SSY,новинка Даша,0,0,0,0,0,...,0,0,0,1,0,None,74,850.0,139.19,148


In [74]:
sbm['                      '] = None

In [75]:
sbm

,Артикул,Название,Номенклатура,TM,Статус,20-February-2024,21-February-2024,22-February-2024,23-February-2024,24-February-2024,...,05-April-2024,06-April-2024,07-April-2024,08-April-2024,,Остаток,Цена,Наценка,Остаток на РЦ,
0,20501,Массажер для шеи и плеч,11111,SSY,текущий,0,0,0,0,0,...,0,0,0,0,None,302,2240.0,77.14,605,None
1,A5501,Камень точильный 6000/1000,99582,Shiny Kitchen,отказ,0,0,0,0,0,...,0,0,1,0,None,173,1410.0,94.04,348,None
2,A5502,Камень точильный 400/1000 и 3000/8000,99093,Shiny Kitchen,отказ,0,0,0,0,0,...,0,0,0,1,None,150,2270.0,85.29,303,None
3,G0155,"Сумка для инструментов, форма 1",11006,SSY,архив,0,0,0,0,0,...,0,0,0,0,None,0,3399.0,100.74,0,None
4,G0156,"Сумка для инструментов, форма 2",12005,SSY,отказ,0,0,0,0,0,...,0,0,0,0,None,0,3599.0,132.23,2,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
986,G0708,"Тактический ошейник, бежевый, XL",25500,SSY,новинка Даша,0,0,0,0,0,...,0,0,0,0,None,98,850.0,139.19,197,None
987,27751,"Утяжелители спортивные черные, 2x2,0 кг.",25406,Shark Fit,текущий Дарья,0,0,0,0,0,...,0,0,1,0,None,108,1100.0,318.07,259,None
988,G0284,"Наколенники с медной нитью, L",25526,Shark Fit,текущий Дарья,0,0,0,0,0,...,0,0,1,0,None,478,1190.0,231.66,959,None
989,G0702,"Тактический ошейник, черный, XL",25494,SSY,новинка Даша,0,0,0,0,0,...,0,0,1,0,None,74,850.0,139.19,148,None


In [76]:
sbm_22

,Артикул,Название,Номенклатура,TM,Статус,20-February-2024,21-February-2024,22-February-2024,23-February-2024,24-February-2024,...,тарифная ставка в рублях_ннп,Итого логистика_ннп,обработка платежей_ннп,налог на пр,итого налог,% налога,прибыль,Наценка,Остаток на РЦ,
0,20501,Массажер для шеи и плеч,11111,SSY,текущий,0,0,0,0,0,...,0.0,108.000000,28.0000,185.673333,371.346667,16.577976,742.693333,77.14,605,None
1,A5501,Камень точильный 6000/1000,99582,Shiny Kitchen,отказ,0,0,0,0,0,...,0.0,80.333333,17.6250,126.050000,252.100000,17.879433,504.200000,94.04,348,None
2,A5502,Камень точильный 400/1000 и 3000/8000,99093,Shiny Kitchen,отказ,0,0,0,0,0,...,0.0,109.000000,28.3750,196.930000,393.860000,17.350661,787.720000,85.29,303,None
3,G0155,"Сумка для инструментов, форма 1",11006,SSY,архив,0,0,0,0,0,...,0.0,149.958333,42.4875,317.737500,635.475000,18.695940,1270.950000,100.74,0,None
4,G0156,"Сумка для инструментов, форма 2",12005,SSY,отказ,0,0,0,0,0,...,0.0,158.291667,44.9875,371.749167,743.498333,20.658470,1486.996667,132.23,2,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985,50187,"Швабра для окон, длинная",25479,Birdhouse,текущий Дарья,0,0,0,0,0,...,0.0,61.666667,10.6250,101.765000,203.530000,23.944706,407.060000,266.64,12,None
986,G0708,"Тактический ошейник, бежевый, XL",25500,SSY,новинка Даша,0,0,0,0,0,...,0.0,61.666667,10.6250,86.011667,172.023333,20.238039,344.046667,139.19,197,None
987,27751,"Утяжелители спортивные черные, 2x2,0 кг.",25406,Shark Fit,текущий Дарья,0,0,0,0,0,...,0.0,70.000000,13.7500,137.718333,275.436667,25.039697,550.873333,318.07,259,None
988,G0284,"Наколенники с медной нитью, L",25526,Shark Fit,текущий Дарья,0,0,0,0,0,...,0.0,73.000000,14.8750,140.365000,280.730000,23.590756,561.460000,231.66,959,None


In [77]:
dfn = None

dfn = sbm_22[['Артикул', 'прибыль', 'Себестоимость', 'Наценка', dt_col(sbm_22)[-1]]]\
     .rename(columns={dt_col(sbm_22)[-1]: 'Продажи'})

dfn = dfn[(dfn['Себестоимость'].astype(str).isin(['0', 'nan', '0.0'])==False)\
        &(dfn['Продажи']>0)]\
     .reset_index(drop=True)

ITOG_NAC = round((((dfn['прибыль'] * dfn['Продажи']).sum() / (dfn['Себестоимость'] *  dfn['Продажи']).sum()) * 100), 2)

dfn = None

In [78]:
ITOG_NAC 

114.95

In [79]:
date_filter = [y.strftime('%d-%B-%Y') for y in sorted([pd.to_datetime(x) for x in dt_col(sbm)])][-1]

sbm['Выручка'] = sbm['Цена'] * sbm[date_filter] 

sbm['Выручка'].loc[len(sbm)-1] = 0
sbm['Выручка'] = sbm['Выручка'].astype(float)
s = sbm['Выручка'].sum()
sbm['Доля выручки'] = (sbm['Выручка'] * 100) / s
sbm['Выручка'].loc[len(sbm)-1] = s
sbm['Итого наценка'] = None
# e = sbm['Итого наценка'].sum()
sbm['Итого наценка'].loc[len(sbm)-1] = ITOG_NAC 

In [80]:
sbm

,Артикул,Название,Номенклатура,TM,Статус,20-February-2024,21-February-2024,22-February-2024,23-February-2024,24-February-2024,...,08-April-2024,,Остаток,Цена,Наценка,Остаток на РЦ,,Выручка,Доля выручки,Итого наценка
0,20501,Массажер для шеи и плеч,11111,SSY,текущий,0,0,0,0,0,...,0,None,302,2240.0,77.14,605,None,0.0,0.000000,None
1,A5501,Камень точильный 6000/1000,99582,Shiny Kitchen,отказ,0,0,0,0,0,...,0,None,173,1410.0,94.04,348,None,0.0,0.000000,None
2,A5502,Камень точильный 400/1000 и 3000/8000,99093,Shiny Kitchen,отказ,0,0,0,0,0,...,1,None,150,2270.0,85.29,303,None,2270.0,1.650513,None
3,G0155,"Сумка для инструментов, форма 1",11006,SSY,архив,0,0,0,0,0,...,0,None,0,3399.0,100.74,0,None,0.0,0.000000,None
4,G0156,"Сумка для инструментов, форма 2",12005,SSY,отказ,0,0,0,0,0,...,0,None,0,3599.0,132.23,2,None,0.0,0.000000,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
986,G0708,"Тактический ошейник, бежевый, XL",25500,SSY,новинка Даша,0,0,0,0,0,...,0,None,98,850.0,139.19,197,None,0.0,0.000000,None
987,27751,"Утяжелители спортивные черные, 2x2,0 кг.",25406,Shark Fit,текущий Дарья,0,0,0,0,0,...,0,None,108,1100.0,318.07,259,None,0.0,0.000000,None
988,G0284,"Наколенники с медной нитью, L",25526,Shark Fit,текущий Дарья,0,0,0,0,0,...,0,None,478,1190.0,231.66,959,None,0.0,0.000000,None
989,G0702,"Тактический ошейник, черный, XL",25494,SSY,новинка Даша,0,0,0,0,0,...,0,None,74,850.0,139.19,148,None,0.0,0.000000,None


### В резерве

In [81]:
vr = pd.read_excel(path_vr)
vr['Артикул'] = vr['Артикул'].astype(str)

sbm = sbm.merge(vr, on='Артикул', how='left')

sbm = sbm[list(sbm.columns[:sbm.columns.get_loc('Остаток на РЦ')+1]) + ['В резерве'] + list(sbm.columns[sbm.columns.get_loc('Остаток на РЦ')+1:-1])]



In [82]:
# sber_orders_metrics = sber_orders_metrics.drop_duplicates().reset_index(drop=True)

In [83]:
print(len(sber_orders_metrics) - len(list(set(sber_orders_metrics['Артикул']))))
print()
sber_orders_metrics

0



,Артикул,Название,Номенклатура,TM,20-February-2023,21-February-2023,22-February-2023,23-February-2023,24-February-2023,25-February-2023,...,30-March-2024,31-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,06-April-2024,07-April-2024,08-April-2024
0,20501,Массажер для шеи и плеч,11111,SSY,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,A5501,Камень точильный 6000/1000,99582,Shiny Kitchen,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,1,0
2,A5502,Камень точильный 400/1000 и 3000/8000,99093,Shiny Kitchen,0,0,0,0,0,0,...,0,0,0,0,1,2,0,0,0,1
3,G0155,"Сумка для инструментов, форма 1",11006,SSY,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,G0156,"Сумка для инструментов, форма 2",12005,SSY,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985,50187,"Швабра для окон, длинная",25479,Birdhouse,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
986,G0708,"Тактический ошейник, бежевый, XL",25500,SSY,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0
987,27751,"Утяжелители спортивные черные, 2x2,0 кг.",25406,Shark Fit,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
988,G0284,"Наколенники с медной нитью, L",25526,Shark Fit,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [84]:
print(len(stocks_sber_metrics) - len(list(set(stocks_sber_metrics['Артикул']))))
print()
stocks_sber_metrics

0



,Артикул,21-February-2023,28-February-2023,01-March-2023,03-March-2023,06-March-2023,07-March-2023,09-March-2023,13-March-2023,14-March-2023,...,27-March-2024,28-March-2024,29-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,08-April-2024,09-April-2024
0,20501,10,10,10,10,10,0,0,0,0,...,333,333,333,303,302,302,302,302,302,302
1,A5501,10,10,10,10,10,10,10,10,10,...,104,179,178,177,176,176,176,175,174,173
2,A5502,10,10,10,10,10,10,10,10,10,...,178,176,174,173,173,160,159,159,158,150
3,G0155,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,G0156,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1092,50975,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1093,50372,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1094,G0641,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1095,G0337,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [85]:
print(len(sber_price_metrics) - len(list(set(sber_price_metrics['Артикул']))))
print()
sber_price_metrics

0



,Артикул,21-February-2023,28-February-2023,01-March-2023,03-March-2023,06-March-2023,07-March-2023,09-March-2023,13-March-2023,14-March-2023,...,30-March-2024,31-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,06-April-2024,07-April-2024,08-April-2024
0,20501,3190,3190,3190,3190,3190,3190,3190,3190,3190,...,2240,2240,2240,2240,2240,2240,2240,2240,2240,2240
1,A5501,1799,1799,1799,1799,1799,1799,1799,1799,1799,...,1410,1410,1410,1410,1410,1410,1410,1410,1410,1410
2,A5502,2215,2215,2215,2215,2215,2215,2215,2215,2215,...,2270,2270,2270,2270,2270,2270,2270,2270,2270,2270
3,G0155,3399,3399,3399,3399,3399,3399,3399,3399,3399,...,3399,3399,3399,3399,3399,3399,3399,3399,3399,3399
4,G0156,3599,3599,3599,3599,3599,3599,3599,3599,3599,...,3599,3599,3599,3599,3599,3599,3599,3599,3599,3599
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1092,G0337,0,0,0,0,0,0,0,0,0,...,0,0,0,1212,1212,1212,1212,1212,1212,1212
1093,B0164,0,0,0,0,0,0,0,0,0,...,0,0,0,1060,1060,1060,1060,1060,1060,1060
1094,50372,0,0,0,0,0,0,0,0,0,...,0,0,0,635,635,635,635,635,635,635
1095,50976,0,0,0,0,0,0,0,0,0,...,0,0,0,399,399,399,399,399,399,399


In [86]:
print(len(sbm_22) - len(list(set(sbm_22['Артикул']))))
print()
sbm_22

0



,Артикул,Название,Номенклатура,TM,Статус,20-February-2024,21-February-2024,22-February-2024,23-February-2024,24-February-2024,...,тарифная ставка в рублях_ннп,Итого логистика_ннп,обработка платежей_ннп,налог на пр,итого налог,% налога,прибыль,Наценка,Остаток на РЦ,
0,20501,Массажер для шеи и плеч,11111,SSY,текущий,0,0,0,0,0,...,0.0,108.000000,28.0000,185.673333,371.346667,16.577976,742.693333,77.14,605,None
1,A5501,Камень точильный 6000/1000,99582,Shiny Kitchen,отказ,0,0,0,0,0,...,0.0,80.333333,17.6250,126.050000,252.100000,17.879433,504.200000,94.04,348,None
2,A5502,Камень точильный 400/1000 и 3000/8000,99093,Shiny Kitchen,отказ,0,0,0,0,0,...,0.0,109.000000,28.3750,196.930000,393.860000,17.350661,787.720000,85.29,303,None
3,G0155,"Сумка для инструментов, форма 1",11006,SSY,архив,0,0,0,0,0,...,0.0,149.958333,42.4875,317.737500,635.475000,18.695940,1270.950000,100.74,0,None
4,G0156,"Сумка для инструментов, форма 2",12005,SSY,отказ,0,0,0,0,0,...,0.0,158.291667,44.9875,371.749167,743.498333,20.658470,1486.996667,132.23,2,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985,50187,"Швабра для окон, длинная",25479,Birdhouse,текущий Дарья,0,0,0,0,0,...,0.0,61.666667,10.6250,101.765000,203.530000,23.944706,407.060000,266.64,12,None
986,G0708,"Тактический ошейник, бежевый, XL",25500,SSY,новинка Даша,0,0,0,0,0,...,0.0,61.666667,10.6250,86.011667,172.023333,20.238039,344.046667,139.19,197,None
987,27751,"Утяжелители спортивные черные, 2x2,0 кг.",25406,Shark Fit,текущий Дарья,0,0,0,0,0,...,0.0,70.000000,13.7500,137.718333,275.436667,25.039697,550.873333,318.07,259,None
988,G0284,"Наколенники с медной нитью, L",25526,Shark Fit,текущий Дарья,0,0,0,0,0,...,0.0,73.000000,14.8750,140.365000,280.730000,23.590756,561.460000,231.66,959,None


In [87]:
print(len(sbm) - len(list(set(sbm['Артикул']))))
print()
sbm

0



,Артикул,Название,Номенклатура,TM,Статус,20-February-2024,21-February-2024,22-February-2024,23-February-2024,24-February-2024,...,,Остаток,Цена,Наценка,Остаток на РЦ,В резерве,,Выручка,Доля выручки,Итого наценка
0,20501,Массажер для шеи и плеч,11111,SSY,текущий,0,0,0,0,0,...,None,302,2240.0,77.14,605,NaN,None,0.0,0.000000,None
1,A5501,Камень точильный 6000/1000,99582,Shiny Kitchen,отказ,0,0,0,0,0,...,None,173,1410.0,94.04,348,1.0,None,0.0,0.000000,None
2,A5502,Камень точильный 400/1000 и 3000/8000,99093,Shiny Kitchen,отказ,0,0,0,0,0,...,None,150,2270.0,85.29,303,2.0,None,2270.0,1.650513,None
3,G0155,"Сумка для инструментов, форма 1",11006,SSY,архив,0,0,0,0,0,...,None,0,3399.0,100.74,0,NaN,None,0.0,0.000000,None
4,G0156,"Сумка для инструментов, форма 2",12005,SSY,отказ,0,0,0,0,0,...,None,0,3599.0,132.23,2,NaN,None,0.0,0.000000,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
986,G0708,"Тактический ошейник, бежевый, XL",25500,SSY,новинка Даша,0,0,0,0,0,...,None,98,850.0,139.19,197,NaN,None,0.0,0.000000,None
987,27751,"Утяжелители спортивные черные, 2x2,0 кг.",25406,Shark Fit,текущий Дарья,0,0,0,0,0,...,None,108,1100.0,318.07,259,42.0,None,0.0,0.000000,None
988,G0284,"Наколенники с медной нитью, L",25526,Shark Fit,текущий Дарья,0,0,0,0,0,...,None,478,1190.0,231.66,959,3.0,None,0.0,0.000000,None
989,G0702,"Тактический ошейник, черный, XL",25494,SSY,новинка Даша,0,0,0,0,0,...,None,74,850.0,139.19,148,NaN,None,0.0,0.000000,None


In [88]:
for i in range(len(sbm_22)):
    if len(sbm_22[sbm_22['Артикул']==sbm_22.loc[i]['Артикул']]) > 1:
        display(sbm_22[sbm_22['Артикул']==sbm_22.loc[i]['Артикул']])

In [89]:
sbm_22

,Артикул,Название,Номенклатура,TM,Статус,20-February-2024,21-February-2024,22-February-2024,23-February-2024,24-February-2024,...,тарифная ставка в рублях_ннп,Итого логистика_ннп,обработка платежей_ннп,налог на пр,итого налог,% налога,прибыль,Наценка,Остаток на РЦ,
0,20501,Массажер для шеи и плеч,11111,SSY,текущий,0,0,0,0,0,...,0.0,108.000000,28.0000,185.673333,371.346667,16.577976,742.693333,77.14,605,None
1,A5501,Камень точильный 6000/1000,99582,Shiny Kitchen,отказ,0,0,0,0,0,...,0.0,80.333333,17.6250,126.050000,252.100000,17.879433,504.200000,94.04,348,None
2,A5502,Камень точильный 400/1000 и 3000/8000,99093,Shiny Kitchen,отказ,0,0,0,0,0,...,0.0,109.000000,28.3750,196.930000,393.860000,17.350661,787.720000,85.29,303,None
3,G0155,"Сумка для инструментов, форма 1",11006,SSY,архив,0,0,0,0,0,...,0.0,149.958333,42.4875,317.737500,635.475000,18.695940,1270.950000,100.74,0,None
4,G0156,"Сумка для инструментов, форма 2",12005,SSY,отказ,0,0,0,0,0,...,0.0,158.291667,44.9875,371.749167,743.498333,20.658470,1486.996667,132.23,2,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985,50187,"Швабра для окон, длинная",25479,Birdhouse,текущий Дарья,0,0,0,0,0,...,0.0,61.666667,10.6250,101.765000,203.530000,23.944706,407.060000,266.64,12,None
986,G0708,"Тактический ошейник, бежевый, XL",25500,SSY,новинка Даша,0,0,0,0,0,...,0.0,61.666667,10.6250,86.011667,172.023333,20.238039,344.046667,139.19,197,None
987,27751,"Утяжелители спортивные черные, 2x2,0 кг.",25406,Shark Fit,текущий Дарья,0,0,0,0,0,...,0.0,70.000000,13.7500,137.718333,275.436667,25.039697,550.873333,318.07,259,None
988,G0284,"Наколенники с медной нитью, L",25526,Shark Fit,текущий Дарья,0,0,0,0,0,...,0.0,73.000000,14.8750,140.365000,280.730000,23.590756,561.460000,231.66,959,None


In [90]:
sbm = sbm[list(sbm.columns[:sbm.columns.get_loc(dt_col(sbm)[0])]) + 
          list(map(lambda x: x.strftime('%d-%B-%Y'), sorted(list(map(lambda x: pd.to_datetime(x), dt_col(sbm)))))) + 
          list(sbm.columns[sbm.columns.get_loc(dt_col(sbm)[-1]) + 1:])]

In [91]:
sbm

,Артикул,Название,Номенклатура,TM,Статус,20-February-2024,21-February-2024,22-February-2024,23-February-2024,24-February-2024,...,,Остаток,Цена,Наценка,Остаток на РЦ,В резерве,,Выручка,Доля выручки,Итого наценка
0,20501,Массажер для шеи и плеч,11111,SSY,текущий,0,0,0,0,0,...,None,302,2240.0,77.14,605,NaN,None,0.0,0.000000,None
1,A5501,Камень точильный 6000/1000,99582,Shiny Kitchen,отказ,0,0,0,0,0,...,None,173,1410.0,94.04,348,1.0,None,0.0,0.000000,None
2,A5502,Камень точильный 400/1000 и 3000/8000,99093,Shiny Kitchen,отказ,0,0,0,0,0,...,None,150,2270.0,85.29,303,2.0,None,2270.0,1.650513,None
3,G0155,"Сумка для инструментов, форма 1",11006,SSY,архив,0,0,0,0,0,...,None,0,3399.0,100.74,0,NaN,None,0.0,0.000000,None
4,G0156,"Сумка для инструментов, форма 2",12005,SSY,отказ,0,0,0,0,0,...,None,0,3599.0,132.23,2,NaN,None,0.0,0.000000,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
986,G0708,"Тактический ошейник, бежевый, XL",25500,SSY,новинка Даша,0,0,0,0,0,...,None,98,850.0,139.19,197,NaN,None,0.0,0.000000,None
987,27751,"Утяжелители спортивные черные, 2x2,0 кг.",25406,Shark Fit,текущий Дарья,0,0,0,0,0,...,None,108,1100.0,318.07,259,42.0,None,0.0,0.000000,None
988,G0284,"Наколенники с медной нитью, L",25526,Shark Fit,текущий Дарья,0,0,0,0,0,...,None,478,1190.0,231.66,959,3.0,None,0.0,0.000000,None
989,G0702,"Тактический ошейник, черный, XL",25494,SSY,новинка Даша,0,0,0,0,0,...,None,74,850.0,139.19,148,NaN,None,0.0,0.000000,None


In [92]:
def weekly_amount1(data):
    
    data1 = data.copy()

    z = dt_col(data)

    for i in range(len(z)):
        if pd.to_datetime(z[i]).dayofweek == 6 and i > 5:
            data1.insert((data1.columns.get_loc(z[i]))+1, f"{pd.to_datetime(z[i-6]):%d-%B}-{pd.to_datetime(z[i]):%d-%B}", 0)



    data = data1

    R = []
    for i in range(len(list(data.columns))):
        try:
            if len((re.findall('(\d+)',list(data.columns)[i]))[1]) != 4:
                R.append(list(data.columns)[i])

        except:
            None

    for row in R:
        T = []
        for i in range(len(data)):
            T.append((data.iloc[i, data.columns.get_loc(row)-7:data.columns.get_loc(row)]).sum())

        data[row] = T
        
    
    data['Артикул'] = data['Артикул'].astype(str)
        
    return data

In [93]:
sbm_col = list(sbm.columns[:5])+dt_col(sbm)[-31:]+list(sbm.columns[sbm.columns.get_loc(dt_col(sbm)[-1])+1:])

In [94]:
sbm = weekly_amount1(sbm)

In [95]:
sbm.columns

Index(['Артикул', 'Название', 'Номенклатура', 'TM', 'Статус',
       '20-February-2024', '21-February-2024', '22-February-2024',
       '23-February-2024', '24-February-2024', '25-February-2024',
       '26-February-2024', '27-February-2024', '28-February-2024',
       '29-February-2024', '01-March-2024', '02-March-2024', '03-March-2024',
       '26-February-03-March', '04-March-2024', '05-March-2024',
       '06-March-2024', '07-March-2024', '08-March-2024', '09-March-2024',
       '10-March-2024', '04-March-10-March', '11-March-2024', '12-March-2024',
       '13-March-2024', '14-March-2024', '15-March-2024', '16-March-2024',
       '17-March-2024', '11-March-17-March', '18-March-2024', '19-March-2024',
       '20-March-2024', '21-March-2024', '22-March-2024', '23-March-2024',
       '24-March-2024', '18-March-24-March', '25-March-2024', '26-March-2024',
       '27-March-2024', '28-March-2024', '29-March-2024', '30-March-2024',
       '31-March-2024', '25-March-31-March', '01-April-20

## Явный подсчет наценки итоговой

In [96]:
# sbm['Итого наценка'].loc[len(sbm)-1] = round(sum(\
#                    [x for x in list(sbm['Итого наценка'][:-1]) if str(x) not in ['inf', 'nan']]), 2)

In [97]:
sbm

,Артикул,Название,Номенклатура,TM,Статус,20-February-2024,21-February-2024,22-February-2024,23-February-2024,24-February-2024,...,,Остаток,Цена,Наценка,Остаток на РЦ,В резерве,,Выручка,Доля выручки,Итого наценка
0,20501,Массажер для шеи и плеч,11111,SSY,текущий,0,0,0,0,0,...,None,302,2240.0,77.14,605,NaN,None,0.0,0.000000,None
1,A5501,Камень точильный 6000/1000,99582,Shiny Kitchen,отказ,0,0,0,0,0,...,None,173,1410.0,94.04,348,1.0,None,0.0,0.000000,None
2,A5502,Камень точильный 400/1000 и 3000/8000,99093,Shiny Kitchen,отказ,0,0,0,0,0,...,None,150,2270.0,85.29,303,2.0,None,2270.0,1.650513,None
3,G0155,"Сумка для инструментов, форма 1",11006,SSY,архив,0,0,0,0,0,...,None,0,3399.0,100.74,0,NaN,None,0.0,0.000000,None
4,G0156,"Сумка для инструментов, форма 2",12005,SSY,отказ,0,0,0,0,0,...,None,0,3599.0,132.23,2,NaN,None,0.0,0.000000,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
986,G0708,"Тактический ошейник, бежевый, XL",25500,SSY,новинка Даша,0,0,0,0,0,...,None,98,850.0,139.19,197,NaN,None,0.0,0.000000,None
987,27751,"Утяжелители спортивные черные, 2x2,0 кг.",25406,Shark Fit,текущий Дарья,0,0,0,0,0,...,None,108,1100.0,318.07,259,42.0,None,0.0,0.000000,None
988,G0284,"Наколенники с медной нитью, L",25526,Shark Fit,текущий Дарья,0,0,0,0,0,...,None,478,1190.0,231.66,959,3.0,None,0.0,0.000000,None
989,G0702,"Тактический ошейник, черный, XL",25494,SSY,новинка Даша,0,0,0,0,0,...,None,74,850.0,139.19,148,NaN,None,0.0,0.000000,None


In [98]:
rc_irc = list(sbm[['Артикул']]\
         .merge(pd.read_excel(path_downloads + 'РЦ Иркутск.xlsx', 
                dtype={'Артикул': 'str'}).groupby('Артикул')['Остаток РЦ Иркутск'].sum()\
                .reset_index(drop=False), 
                on='Артикул', 
                how='left')['Остаток РЦ Иркутск'])

sbm.insert(sbm.columns.get_loc('Остаток на РЦ')+1, 'Остаток РЦ Иркутск', rc_irc)

### Добавление акции 

In [99]:
ac_columns = []
for i in range(len(acc_name)):
    acc = pd.read_excel(path_downloads + acc_path[i],
                        sheet_name='Параметры акции',
                        skiprows=2,
                       )
    if 'Unnamed: 8' not in acc.columns:
        acc = acc[['Unnamed: 1', 'Unnamed: 12']] 
    else:
        acc = acc[['Unnamed: 1', 'Unnamed: 8']]
    acc = acc.iloc[1:, :].dropna().reset_index(drop=True)
    acc.columns = ['Номенклатура', 'Цена']
    acc['Цена'] = acc['Цена'].astype(float)
    if len(acc) - len(list(set(acc['Номенклатура']))) != 0:
        print('error')
        int('error')
    sbm_acc['Номенклатура'] = sbm_acc['Номенклатура'].astype(str)
    if len(sbm_acc) - len(list(set(sbm_acc['Номенклатура']))) != 0:
        print('error')
        int('error')
    acc = sbm_acc[['Артикул', 'Номенклатура', 'Себестоимость', 'тарифная ставка']]\
          .merge(acc, on='Номенклатура', how='outer').dropna().reset_index(drop=True)
    acc = sbm_nac(acc)
    acc = acc[['Артикул', 'Цена', 'Наценка']]
    acc.columns = ['Артикул', f'Цена_{acc_name[i]}', f'Наценка_{acc_name[i]}']
    ac_columns.append(f'Цена_{acc_name[i]}')
    ac_columns.append(f'Наценка_{acc_name[i]}')
    sbm = sbm.merge(acc, on='Артикул', how='left')
    if len(sbm) - len(list(set(sbm['Артикул']))) != 0:
        print('error')
        int('error')
#     display(acc)
        

In [100]:
pd.DataFrame(ac_columns, 
             columns=['Col_for_not_del']).to_excel(path_downloads + 'Col_for_not_del_sb.xlsx', index=False)

## прибыль с единицы товара

In [101]:
sbm = sbm.merge(sbm_22[['Артикул', 
                        'прибыль']]\
              .rename(columns={'прибыль': 'прибыль с единицы товара'}),
              on='Артикул', 
              how='left') 
sbm['прибыль с единицы товара'] = sbm['прибыль с единицы товара'].round(2)

## Создание таблицы с изменением цен

In [102]:
if len(sber_price_metrics) - len(list(set(sber_price_metrics['Артикул']))) !=0:
    print('error')
    int('error')
else:
    sber_price_difference = sber_price_metrics[['Артикул']+dt_col(sber_price_metrics)[-6:]].copy()
    cols_for_analysis = list(sber_price_difference.columns)[1:][::-1]
    for i in range(len(cols_for_analysis)-1):
        sber_price_difference['Изм. цены ' + pd.to_datetime(cols_for_analysis[i])\
                            .strftime('%d.%m')] = sber_price_difference[cols_for_analysis[i]]\
                                                    - sber_price_difference[cols_for_analysis[i+1]]
#         ozon_price_difference['Цена '+ pd.to_datetime(cols_for_analysis[i])\
#                             .strftime('%d.%m')] = ozon_price_difference[cols_for_analysis[i]]
    sber_price_difference = sber_price_difference.drop(cols_for_analysis, axis=1)
    sber_price_difference = sber_price_difference[['Артикул']\
                                             + list(sber_price_difference.columns[1:])[::-1]]
    try:
        C_price = pd.read_excel(path_downloads + 'Целевая цена.xlsx',
                                sheet_name='sber', 
                                dtype={'Артикул': 'str'})
        sber_price_difference = sber_price_difference.merge(C_price, on='Артикул', how='left')
    except:
        None
    sber_price_difference.to_excel(path_downloads + 'Изменения цены сбер.xlsx', index=False)

In [103]:
sber_orders_metrics = wwm.rename_data_from_reestr(sber_orders_metrics, 
                                                  'Название', 
                                                  reestr=reestr)
sbm_22 = wwm.rename_data_from_reestr(sbm_22, 
                                     'Название', 
                                      reestr=reestr)
sbm = wwm.rename_data_from_reestr(sbm, 
                                  'Название',  
                                  reestr=reestr)

In [104]:
sber_orders_metrics.to_excel(path_sber_orders_metrics, index=False)
stocks_sber_metrics.to_excel(path_stocks_sber_metrics, index=False)
sber_price_metrics.to_excel(path_sber_price_metrics, index=False)

In [105]:
sbm[sbm['Артикул']=='28812']

,Артикул,Название,Номенклатура,TM,Статус,20-February-2024,21-February-2024,22-February-2024,23-February-2024,24-February-2024,...,Наценка_ЗОО Распродажа апреля_y,Цена_Время обновлений. Все для ремонта_02.04-31.05_Сикдка от 10%,Наценка_Время обновлений. Все для ремонта_02.04-31.05_Сикдка от 10%,Цена_Событийное промо: Пасха_18.04-05.05,Наценка_Событийное промо: Пасха_18.04-05.05,Цена_Скидки на российскую косметику 08.04-21.04,Наценка_Скидки на российскую косметику 08.04-21.04,Цена_Распродажа тренажеров и товаров для фитнеса_15.04-12.05_Скидка от 15%,Наценка_Распродажа тренажеров и товаров для фитнеса_15.04-12.05_Скидка от 15%,прибыль с единицы товара
744,28812,Напульсник эластичный,25280,Shark Fit,текущий Дарья,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,207.23


In [106]:
sbm['Наценка']

0       77.14
1       94.04
2       85.29
3      100.74
4      132.23
        ...  
986    139.19
987    318.07
988    231.66
989    139.19
990      None
Name: Наценка, Length: 991, dtype: object

In [107]:
sbm['Итого наценка']

0        None
1        None
2        None
3        None
4        None
        ...  
986      None
987      None
988      None
989      None
990    114.95
Name: Итого наценка, Length: 991, dtype: object

In [108]:
pda(sber_orders_metrics)

Уникальных артикулов: 990
Всего артикулов: 990





In [109]:
pda(stocks_sber_metrics)

Уникальных артикулов: 1097
Всего артикулов: 1097





In [110]:
pda(sber_price_metrics)

Уникальных артикулов: 1097
Всего артикулов: 1097





In [111]:
pda(sbm_22)

Уникальных артикулов: 990
Всего артикулов: 990





In [112]:
pda(sbm)

Уникальных артикулов: 991
Всего артикулов: 991





## Сохранение наценки

In [113]:
sb_markup_metrics = pd.read_excel(wwm.path_sb_markup_metrics,
                                  dtype={'Артикул': 'str'})
sb_markup_metrics = sb_markup_metrics[sb_markup_metrics['Артикул'].astype(str)!='nan']\
                    .reset_index(drop=True)
if len(sb_markup_metrics) - len(list(set(sb_markup_metrics['Артикул']))) != 0:
    print('Неуникальные артикулы в sb_markup_metrics')
    exit()
sb_markup_new = sbm_22[['Артикул', 'Наценка']]\
                .rename(columns={'Наценка': const.dt_now.strftime('%d-%B-%Y')}).copy()
if len(sb_markup_new) - len(list(set(sb_markup_new['Артикул']))) != 0:
    print('Неуникальные артикулы в sb_markup_new')
    exit()
col = wwm.dt_col(sb_markup_new)[0]
for i in range(len(sb_markup_new)):
    art = sb_markup_new.loc[i]['Артикул']
    if art in list(set(sber_price_metrics['Артикул'])):
        if sber_price_metrics[sber_price_metrics['Артикул']==art][sber_price_metrics.columns[-1]].values[0] == 0:
            sb_markup_new.loc[i, col] = None
if col in sb_markup_metrics.columns:
    sb_markup_metrics = sb_markup_metrics.drop(col, axis=1)
    sb_markup_metrics = sb_markup_metrics.merge(sb_markup_new, on='Артикул', how='outer')
else:
    sb_markup_metrics = sb_markup_metrics.merge(sb_markup_new, on='Артикул', how='outer')
sb_markup_metrics = sb_markup_metrics[sb_markup_metrics['Артикул'].astype(str)!='nan']\
                    .reset_index(drop=True)
if len(sb_markup_metrics) - len(list(set(sb_markup_metrics['Артикул']))) != 0:
    print('Неуникальные артикулы в sb_markup_metrics')
    exit()
sb_markup_metrics.to_excel(wwm.path_sb_markup_metrics, index=False)

In [114]:
sbm[dt_col(sbm)[-10:]]

,30-March-2024,31-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,06-April-2024,07-April-2024,08-April-2024
0,1,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,1,0
2,0,0,0,0,1,2,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
986,0,0,0,0,2,0,0,0,0,0
987,0,0,0,0,0,0,0,0,1,0
988,0,0,0,0,0,0,0,0,1,0
989,0,0,0,0,0,0,0,0,1,0


In [115]:
def unloading():
    
    with pd.ExcelWriter(path_algoritm + "Все файлы для ежедневной презентации/full_SBER_df.xlsx") as writer:


        sber_orders_metrics.to_excel(writer, sheet_name="sber_orders_metrics", index=False)
        for column in sber_orders_metrics:
            column_width = max(sber_orders_metrics[column].astype(str).map(len).max(), len(column))
            col_idx = sber_orders_metrics.columns.get_loc(column)
            writer.sheets['sber_orders_metrics'].set_column(col_idx, col_idx, column_width)



        stocks_sber_metrics.to_excel(writer, sheet_name="stocks_sber_metrics", index=False)
        for column in stocks_sber_metrics:
            column_width = max(stocks_sber_metrics[column].astype(str).map(len).max(), len(column))
            col_idx = stocks_sber_metrics.columns.get_loc(column)
            writer.sheets['stocks_sber_metrics'].set_column(col_idx, col_idx, column_width)



        sber_price_metrics.to_excel(writer, sheet_name="sber_price_metrics", index=False)
        for column in sber_price_metrics:
            column_width = max(sber_price_metrics[column].astype(str).map(len).max(), len(column))
            col_idx = sber_price_metrics.columns.get_loc(column)
            writer.sheets['sber_price_metrics'].set_column(col_idx, col_idx, column_width)







        sbm_22.to_excel(writer, sheet_name="sber_full_features_metrics", index=False)
        for column in sbm_22:
            column_width = max(sbm_22[column].astype(str).map(len).max() + 3, len(column))
            col_idx = sbm_22.columns.get_loc(column)
            writer.sheets['sber_full_features_metrics'].set_column(col_idx, col_idx, column_width)



        sbm.to_excel(writer, sheet_name="sber_full_df", index=False)
        for column in sbm:
            column_width = max(sbm[column].astype(str).map(len).max() + 3, len(column))
            col_idx = sbm.columns.get_loc(column)
            writer.sheets['sber_full_df'].set_column(col_idx, col_idx, column_width)



    writer.save()

In [116]:
try:
    unloading()
except:
    unloading()

/Users/dmitriymaslov/anaconda3/envs/work_venv/lib/python3.9/site-packages/xlsxwriter/workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [117]:
list(sbm.columns)

['Артикул',
 'Название',
 'Номенклатура',
 'TM',
 'Статус',
 '20-February-2024',
 '21-February-2024',
 '22-February-2024',
 '23-February-2024',
 '24-February-2024',
 '25-February-2024',
 '26-February-2024',
 '27-February-2024',
 '28-February-2024',
 '29-February-2024',
 '01-March-2024',
 '02-March-2024',
 '03-March-2024',
 '26-February-03-March',
 '04-March-2024',
 '05-March-2024',
 '06-March-2024',
 '07-March-2024',
 '08-March-2024',
 '09-March-2024',
 '10-March-2024',
 '04-March-10-March',
 '11-March-2024',
 '12-March-2024',
 '13-March-2024',
 '14-March-2024',
 '15-March-2024',
 '16-March-2024',
 '17-March-2024',
 '11-March-17-March',
 '18-March-2024',
 '19-March-2024',
 '20-March-2024',
 '21-March-2024',
 '22-March-2024',
 '23-March-2024',
 '24-March-2024',
 '18-March-24-March',
 '25-March-2024',
 '26-March-2024',
 '27-March-2024',
 '28-March-2024',
 '29-March-2024',
 '30-March-2024',
 '31-March-2024',
 '25-March-31-March',
 '01-April-2024',
 '02-April-2024',
 '03-April-2024',
 '04-

In [118]:
exit()

SystemExit: 

/Users/dmitriymaslov/anaconda3/envs/work_venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
